<a href="https://colab.research.google.com/github/Huy-Egoist/Performance-evaluation-of-computer-systems/blob/main/HK151_SPE_Dragon_Protocol.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Indexed Allocation File System

# 1. Team work declaration

## Team
| Student              | Id      |
|-------------------   |---------|
| Nguyễn Minh Nhật     | 2212397 |
|  Dương Gia Huy       | 2211155 |
| Nguyễn Quốc Tuấn Lâm | 2211816 |
| Hà Trần Quang Trung  | 2014874 |
|  Lưu Văn Huy         | 2211199 |
|Nguyễn Vĩ Nhân        | 2212375 |
|Phan Châu Phong       | 2212563 |

## Workmap declaration

| Last name | Student ID | Design Role | Dev Role | Eval Role | Report Role | Ass1 work description (details in text)  | Rating | week 2 outcome | week 3 outcome | week 4 outcome  |
|-----------|------------|-------------|----------|-----------|-------------|------------------------------------------|-----------------|----------------|----------------|-----------------|
| Nguyễn Minh Nhật        |    2212397        | Platoon 1      | yes/no   | yes/no    | yes/no      |                                          | A/B/C           |                |                |                 |
| Dương Gia Huy        |  2211155          | Platoon 1      | yes/no   | yes/no    | yes/no      |                                          | A/B/C           |                |                |                 |
| Nguyễn Quốc Tuấn Lâm        |  2211816          | Platoon 2      | yes/no   | yes/no    | yes/no      |                                          | A/B/C           |                |                |                 |
| Hà Trần Quang Trung        | 2014874           | Platoon 2      | yes/no   | yes/no    | yes/no      |                                          | A/B/C           |                |                |                 |
| Lưu Văn Huy        | 2211199           | Platoon 3      | yes/no   | yes/no    | yes/no      |                                          | A/B/C           |                |                |                 |
| Nguyễn Vĩ Nhân        | 2212375           | Platoon 3      | yes/no   | yes/no    | yes/no      |                                          | A/B/C           |                |                |                 |
| Phan Châu Phong       | 2212563           |   Platoon 3          |  yes/no        | yes/no          | yes/no            |                                          |    A/B/C             |                |                |                 |
|           |            |             |          |           |             |                                          |                 |                |                |                 |


# 2.System design

## System Performance Steps

### 1. Define Goal and System

### 2. Service and Outcomes

### 3. Select Metrics

### 4. List System Parameters

### 5. List Factors to Study

### 6. Evaluation Technique

### 7. Select Workload



## System Component/Module description

# 3. System Implementation



Install prerequisite library


*   simpy



In [39]:
pip install simpy

Import các thư viện cần thiết

In [40]:
import simpy
import random
import numpy as np
import pandas as pd

## System configuration and execution

###1. Thiết kế cấu trúc dữ liệu

####1.1. Class Request

In [41]:
class Request:
    def __init__(self, req_id, arrival_time):
        self.id = req_id
        self.arrival_time = arrival_time
        self.current_level = 1  # Bắt đầu từ Level 1 (Cache)

        # Các biến để thu thập số liệu
        self.completed = False
        self.dropped = False
        self.start_service_time = -1 # -1 nghĩa là chưa được phục vụ
        self.end_service_time = -1   # -1 nghĩa là chưa hoàn thành

    def __repr__(self):
        # Hàm này giúp in ra thông tin request cho dễ gỡ lỗi (debug)
        return f"Request({self.id} @ {self.arrival_time:.2f})"

###1.2. StorageLevel: Mô phỏng tầng 2(SSD) và tầng 3(HDD)

In [42]:
class StorageLevel:
    def __init__(self, env, name, mu, c, K):
        self.env = env
        self.name = name
        self.mu = mu  # Tốc độ phục vụ (service rate)
        self.c = c    # Số server
        self.K = K    # Dung lượng hàng đợi (queue size)

        # Đối tượng SimPy cho c server song song
        self.servers = simpy.Resource(env, capacity=c)

        # Đối tượng SimPy cho hàng đợi FIFO có giới hạn K
        self.queue = simpy.Store(env, capacity=K)

    def __repr__(self):
        return f"StorageLevel({self.name}, mu={self.mu}, c={self.c}, K={self.K})"

Trong bản thiết kế của Platoon 1 (mục 4.1), hệ thống có bốn cấu trúc dữ liệu chính: **Request, StorageLevel, ServerState và Event.**

Tuy nhiên, khi chuyển sang hiện thực bằng SimPy (Platoon 2), có một thay đổi như sau:

**ServerState:** Phần này không cần tạo riêng nữa, vì việc quản lý trạng thái busy/idle của các server đã được **`simpy.Resource`** xử lý sẵn. Còn **`level.servers.count`** và khối lệnh with **`level.servers.request()`** được để quản lý tài nguyên, nên không cần thêm lớp ServerState.

**Event và EventList:** Trong SimPy, hai khái niệm này đã được tích hợp sẵn trong cơ chế mô phỏng sự kiện.Biến **`env = simpy.Environment()`** đóng vai trò là bộ điều phối và quản lý hàng đợi sự kiện (EventList). Các lệnh **`yield env.timeout()`** chính là cách tạo và sắp lịch các sự kiện (Event) một cách tự động, nên không cần phải tự xây dựng danh sách sự kiện (priority queue) như trong bản thiết kế ban đầu.

Vì vậy, ở phần hiện thực (Platoon 2), chỉ cần định nghĩa hai lớp chính là **Request** và **StorageLevel**, tập trung vào phần logic xử lý chính của hệ thống, thay vì phải viết lại những cơ chế mà SimPy đã hỗ trợ.

###2. Biến toàn cục

Tương tự như các cấu trúc dữ liệu, các biến toàn cục được đề xuất trong mục 4.2 của bản thiết kế đã được có sẵn bằng các cơ chế của SimPy và Python:

* **`global current_time`:** Được thay thế bằng `env.now` của SimPy. Biến này tự động theo dõi thời gian mô phỏng.
* **`global EventList`:** Được thay thế bằng `env = simpy.Environment()`. SimPy tự động quản lý hàng đợi sự kiện này.
* **`global Levels`:** Thay vì một `dict` toàn cục, tạo các biến `level2_a`, `level3_a`... và truyền chúng vào các hàm khi cần.
* **`global Stats`:** Sử dụng các danh sách Python (ví dụ: `results_a`, `dropped_a`) để thu thập số liệu.
* **`global SIM_END_TIME`:** Biến `SIM_END_TIME = 1000` (hoặc giá trị khác) được dùng để chạy mô phỏng.

###3. Các hàm chính

####3.1. Tạo các yêu cầu đọc ghi mới

In [43]:
def generate_requests(env, lam, p_hit, level2, level3, results_list, dropped_list, l1_hit_list):
    """Hàm sinh request liên tục theo phân phối Poisson"""
    req_id = 0
    while True:
        # 1. Tính thời gian chờ (inter-arrival time) theo phân phối Mũ
        inter_arrival_time = random.expovariate(lam)

        # 2. Mô phỏng thời gian chờ
        yield env.timeout(inter_arrival_time)

        # 3. Tạo request mới
        req_id += 1
        new_req = Request(req_id, env.now)

        # 4. Đẩy request vào Tầng 1 (Cache)
        env.process(arrival_level_1(env, new_req, p_hit, level2, level3, results_list, dropped_list, l1_hit_list))

####3.2. Xử lí khi một yêu cầu đến tầng hiện tại

In [44]:
def arrival_level_storage(env, req, level, next_level, results_list, dropped_list):
    """
    Xử lý logic khi request đến L2 (SSD) hoặc L3 (HDD)
    """

    # Kiểm tra xem server có rảnh không
    if level.servers.count < level.servers.capacity:
        # Có server rảnh -> Phục vụ ngay
        env.process(begin_service(env, req, level, results_list))

    # Server bận, kiểm tra hàng đợi
    elif len(level.queue.items) < level.K:
        # Hàng đợi còn chỗ -> Xếp hàng
        # Lưu ý: Phải dùng 'yield' ở đây vì .put() là một hoạt động của SimPy
        yield level.queue.put(req)

    # Hàng đợi đầy
    else:
        # Kiểm tra xem có tầng tiếp theo không
        if next_level:
            # Gọi hàm forward để chuyển tới tầng tiếp theo
            env.process(forward(env, req, next_level, results_list, dropped_list))
        else:
            # Đây là L3 và hàng đợi L3 cũng đầy -> Drop
            req.dropped = True
            req.end_service_time = env.now # Ghi nhận thời điểm bị drop
            dropped_list.append(req)

def arrival_level_1(env, req, p_hit, level2, level3, results_list, dropped_list, l1_hit_list):
    """
    Xử lý logic tại L1 (Cache) - Cổng vào hệ thống
    """

    # Giả định thời gian tra cứu cache là không đáng kể (0)
    yield env.timeout(0)
    if random.random() < p_hit:
        # --- CACHE HIT ---
        req.completed = True
        req.end_service_time = env.now
        results_list.append(req)
        l1_hit_list.append(req) # Ghi nhận L1 hit
    else:
        # --- CACHE MISS ---
        # Chuyển xuống Tầng 2 (SSD)
        req.current_level = 2
        # Gọi arrival_level_storage, truyền 'level2' là tầng hiện tại
        # và 'level3' là tầng kế tiếp (next_level)
        env.process(arrival_level_storage(env, req, level2, level3, results_list, dropped_list))

####3.3. và 3.4. Bắt đầu/Kết thúc phục vụ request và xử lí request tiếp theo

In [45]:
def begin_service(env, req, level, results_list):
    """Mô phỏng 1 request chiếm 1 server, được phục vụ, và nhả server ra"""
    # Chiếm 1 server
    with level.servers.request() as server_req:
        yield server_req  # Chờ đến khi có server rảnh

        # ------ BẮT ĐẦU PHỤC VỤ ------
        req.start_service_time = env.now

        # Tính thời gian phục vụ (theo phân phối Mũ)
        service_time = random.expovariate(level.mu)

        # Mô phỏng thời gian phục vụ
        yield env.timeout(service_time)

        # ------ KẾT THÚC PHỤC VỤ ------
        req.end_service_time = env.now
        req.completed = True
        results_list.append(req) # Lưu request đã hoàn thành

    # ------ SERVER ĐÃ RẢNH ------
    # Kiểm tra xem có ai đang chờ trong hàng đợi (queue) không
    if len(level.queue.items) > 0:
        # Lấy request tiếp theo từ hàng đợi
        next_req = yield level.queue.get()

        # Bắt đầu phục vụ request đó
        env.process(begin_service(env, next_req, level, results_list))

####3.5. Chuyển tiếp request tới tầng lưu trữ kế tiếp

In [46]:
def forward(env, req, next_level, results_list, dropped_list):
    """
    Hiện thực hóa Mục 4.3.5: Chuyển tiếp request đến tầng lưu trữ kế tiếp.
    """
    yield env.timeout(0)
    # 1. Cập nhật tầng mới cho request (req.current_level = next_level)
    # Giả định next_level.name là 'SSD' (L2) hoặc 'HDD' (L3)
    if next_level.name == "SSD":
        req.current_level = 2
    elif next_level.name == "HDD":
        req.current_level = 3

    # 2. Lên lịch một sự kiện "arrival" ở tầng tiếp theo (schedule_event)
    # (Chúng ta không cần dòng req.arrival_time_at_level = env.now
    # vì hàm arrival_level_storage sẽ xử lý việc đó)
    env.process(arrival_level_storage(env, req, next_level, None, results_list, dropped_list))

###4. Mô phỏng

####4.1. Workload A: Kịch bản thiên về Đọc (Read-heavy)

In [47]:
print("--- Đang chạy Workload A (Read-heavy) ---")

# 1. Thiết lập tham số
SIM_END_TIME = 1000  # Chạy mô phỏng trong 1000 giây
LAMBDA_A = 100.0       # 100 req/s
P_HIT_A = 0.85         # 85% cache hit

# 2. Khởi tạo môi trường
env_a = simpy.Environment()

# 3. Tạo các tầng lưu trữ
# L2 (SSD): mu=200, c=4, K=100
level2_a = StorageLevel(env_a, "SSD", mu=200.0, c=4, K=100)
# L3 (HDD): mu=50, c=2, K=50
level3_a = StorageLevel(env_a, "HDD", mu=50.0, c=2, K=50)

# 4. Danh sách thu thập kết quả
results_a = []
dropped_a = []
l1_hits_a = []

# 5. Khởi chạy process
env_a.process(generate_requests(env_a, LAMBDA_A, P_HIT_A, level2_a, level3_a,
                                results_a, dropped_a, l1_hits_a))

# 6. Chạy mô phỏng
env_a.run(until=SIM_END_TIME)

print(f"--- Workload A hoàn thành ---")
print(f"Tổng số request đã xử lý (hoàn thành hoặc drop): {len(results_a) + len(dropped_a)}")
print(f"Hoàn thành: {len(results_a)}")
print(f"Bị drop: {len(dropped_a)}")
print(f"Cache Hit L1: {len(l1_hits_a)}")

--- Đang chạy Workload A (Read-heavy) ---
--- Workload A hoàn thành ---
Tổng số request đã xử lý (hoàn thành hoặc drop): 100294
Hoàn thành: 100294
Bị drop: 0
Cache Hit L1: 85474


####4.2. Workload B: Kịch bản thiên về Ghi (Write-heavy)

In [48]:
print("\n--- Đang chạy Workload B (Write-heavy) ---")

# 1. Thiết lập tham số
LAMBDA_B = 200.0       # 200 req/s
P_HIT_B = 0.40         # 40% cache hit

# 2. Khởi tạo môi trường
env_b = simpy.Environment()

# 3. Tạo các tầng lưu trữ
# [cite_start]L2 (SSD): mu=180, c=4, K=100
level2_b = StorageLevel(env_b, "SSD", mu=180.0, c=4, K=100)
# [cite_start]L3 (HDD): mu=40, c=2, K=50
level3_b = StorageLevel(env_b, "HDD", mu=40.0, c=2, K=50)

# 4. Danh sách thu thập kết quả
results_b = []
dropped_b = []
l1_hits_b = []

# 5. Khởi chạy process
env_b.process(generate_requests(env_b, LAMBDA_B, P_HIT_B, level2_b, level3_b,
                                results_b, dropped_b, l1_hits_b))

# 6. Chạy mô phỏng
env_b.run(until=SIM_END_TIME)

print(f"--- Workload B hoàn thành ---")
print(f"Tổng số request đã xử lý (hoàn thành hoặc drop): {len(results_b) + len(dropped_b)}")
print(f"Hoàn thành: {len(results_b)}")
print(f"Bị drop: {len(dropped_b)}")
print(f"Cache Hit L1: {len(l1_hits_b)}")


--- Đang chạy Workload B (Write-heavy) ---
--- Workload B hoàn thành ---
Tổng số request đã xử lý (hoàn thành hoặc drop): 200107
Hoàn thành: 200107
Bị drop: 0
Cache Hit L1: 80412


####4.3. Workload C: Kịch bản (Stress Test)

In [49]:
print("\n---  Đang chạy Workload C (Stress Test)  ---")

# 1. Thiết lập tham số
# Chúng ta sẽ dùng các tham số của Workload B (P_HIT, mu, c, K)
# NHƯNG tăng tốc độ đến (lambda) lên RẤT CAO
# Cụ thể: 1500 req/s. (Lý do: 1500 * (1-0.4) = 900 req/s đến L2,
# trong khi L2 chỉ xử lý được 4*180 = 720 req/s -> L2 sẽ đầy)
LAMBDA_C = 1500.0
P_HIT_C = 0.40         # Giữ nguyên 40% cache hit của Workload B

# 2. Khởi tạo môi trường
env_c = simpy.Environment()

# 3. Tạo các tầng lưu trữ (Giữ nguyên cấu hình của Workload B)
# L2 (SSD): mu=180, c=4, K=100
level2_c = StorageLevel(env_c, "SSD", mu=180.0, c=4, K=100)
# L3 (HDD): mu=40, c=2, K=50
level3_c = StorageLevel(env_c, "HDD", mu=40.0, c=2, K=50)

# 4. Danh sách thu thập kết quả
results_c = []
dropped_c = []
l1_hits_c = []

# 5. Khởi chạy process
env_c.process(generate_requests(env_c, LAMBDA_C, P_HIT_C, level2_c, level3_c,
                                results_c, dropped_c, l1_hits_c))

# 6. Chạy mô phỏng (Vẫn chạy 1000 giây)
env_c.run(until=SIM_END_TIME)

print(f"--- Workload C hoàn thành ---")
print(f"Tổng số request đã xử lý (hoàn thành hoặc drop): {len(results_c) + len(dropped_c)}")
print(f"Hoàn thành: {len(results_c)}")
print(f" Bị drop: {len(dropped_c)} ")
print(f"Cache Hit L1: {len(l1_hits_c)}")


---  Đang chạy Workload C (Stress Test)  ---
--- Workload C hoàn thành ---
Tổng số request đã xử lý (hoàn thành hoặc drop): 1501557
Hoàn thành: 1400076
 Bị drop: 101481 
Cache Hit L1: 599524
